### Simple requirements calculations for estimating the distance between cameras and detection distance

| Symbol      | Units | Meaning                                                                 |
|-------------|-------|-----------------------------------------------------------------|
| $f_{mm}$    | mm    | Lens focal length from datasheet                                    |
| $p_{μm}$    | μm    | Pixel pitch (pixel size) from datasheet                            |
| $p_{mm}$    | mm    | Pixel pitch converted to millimeters ($p_{μm}/1000$)               |
| $f_{px}$    | px    | Effective focal length in pixel units, computed as $f_{mm}/p_{mm}$ |
| $N_x, N_y$  | px    | Horizontal and vertical resolution (e.g. 1920×1080 for 1080p)      |
| $B$         | m     | Baseline, spacing between the two camera centers                   |
| $Z$         | m     | Depth (distance from the camera plane to the object)               |
| $d$         | px    | Disparity = horizontal shift of the object between left and right images |
| $σ_Z$       | m     | Standard deviation of depth estimate (uncertainty in distance)     |
| $σ_d$       | px    | Standard deviation of disparity estimate (uncertainty in disparity) |
| $σ_x$       | px    | Standard deviation of detected object center in one image (pixel uncertainty from YOLO bounding box center or refined estimate) |
| $d_0$       | px    | Expected disparity for a target depth $Z$: $d_0 = f_{px} B / Z$    |


In [72]:
import numpy as np
import pandas as pd

### Converting to focal length in pixels
$$
p_{mm} = \frac{p_{\mu m}}{1000}, \quad f_{px}=\frac{f_{mm}}{p_{mm}}=\frac{f_{mm} \cdot 1000}{p_{\mu m}}
$$

Binning correction where we have k bins per effective pixel. This is equivelant to zooming in.
$$
p_{eff} = kp_{mm}, \quad f_{px,eff} = \frac{f_{mm}}{p_{eff}} = \frac{f_{px}}{k}
$$

### Stero vision relationships
$$
d=\frac{f B}{Z} \Leftarrow\Rightarrow Z = \frac{f B}{d}
$$
This shows that the difference in left and right images is a function of the distance between the two cameras (disparity). 

In [ ]:
# basic utilities to convert 
def fx_from_datasheet(f_mm: float, p_um: float, *, binning: float = 1.0, downscale: float = 1.0) -> float:
    """
    Convert focal length in mm and pixel size in micrometers to focal length in pixels.
    Applies binning and digital downscale factors.
    f_px = (f_mm / p_mm) / (binning * downscale)  where p_mm = p_um / 1000
    """
    if f_mm <= 0 or p_um <= 0:
        raise ValueError("f_mm and p_um must be positive.")
    p_mm = p_um / 1000.0
    f_px = (f_mm / p_mm) / (binning * downscale)
    return f_px

def disparity_expected(f_px: float, B: float, Z: float) -> float:
    """Expected disparity (pixels) at depth Z: d = f_px * B / Z"""
    if any(v <= 0 for v in (f_px, B, Z)):
        raise ValueError("f_px, B, Z must be positive.")
    return f_px * B / Z

def depth_from_disparity(f_px: float, B: float, d: float) -> float:
    """Depth (meters) from disparity (pixels): Z = f_px * B / d"""
    if any(v <= 0 for v in (f_px, B, d)):
        raise ValueError("f_px, B, d must be positive.")
    return f_px * B / d

def sigma_d_from_sigma_x(sigma_x: float) -> float:
    """Disparity std from per-camera center std: sigma_d = sqrt(2) * sigma_x"""
    if sigma_x <= 0:
        raise ValueError("sigma_x must be positive.")
    return np.sqrt(2.0) * sigma_x

def sigmaZ_from_sigmaX(f_px: float, B: float, Z: float, sigma_x: float) -> float:
    """
    Depth std (meters) from per-camera center std (pixels):
    sigma_Z = (Z^2 / (f_px * B)) * sigma_d  with sigma_d = sqrt(2) * sigma_x
    """
    if any(v <= 0 for v in (f_px, B, Z, sigma_x)):
        raise ValueError("f_px, B, Z, sigma_x must be positive.")
    sigma_d = sigma_d_from_sigma_x(sigma_x)
    return (Z**2 / (f_px * B)) * sigma_d

def baseline_for_target_sigmaZ(f_px: float, Z: float, sigma_Z: float, sigma_x: float) -> float:
    """
    Required baseline (meters) to reach a target depth std sigma_Z (meters):
    B = (Z^2 / f_px) * (sqrt(2) * sigma_x / sigma_Z)
    """
    if any(v <= 0 for v in (f_px, Z, sigma_Z, sigma_x)):
        raise ValueError("f_px, Z, sigma_Z, sigma_x must be positive.")
    return (Z**2 / f_px) * (np.sqrt(2.0) * sigma_x / sigma_Z)

def disparity_sanity(d: float, Nx: int, *, min_detectable: float = 4.0, margin_px: int = 20) -> dict:
    """
    Quick checks for disparity magnitude relative to image width.
    Returns info with booleans for 'enough_for_matching' and 'fits_in_frame'.
    """
    if Nx <= 0:
        raise ValueError("Nx must be positive.")
    half_width = Nx / 2.0
    return {
        "enough_for_matching": d >= min_detectable,
        "fits_in_frame": (d / 2.0) < (half_width - margin_px),
        "half_image_width_px": half_width,
    }

### What are some of these symbols?
Disparity limit is sating we need at least $d_min$ pixels to detect/triangulate. This translate into the minimum bounding box size we get from the YOLO classification model.

Accuracy limit says we want to stay within some sort of target depth error $\sigma_Z^{target}$ so we can have a reasonable understanding of how far the target is

Apparent size limit says that the object will span at least $s_{min}$ pixels. This is a function of the objects physical size, $f_px$ of the camera, and distance from the camera. This is given by

$$
Z_{max size} = \frac{f_{px}S}{s_{min}}
$$

Where S is the objects physical size and $s$ is the objects image size, $s=\frac{f_{px}S}{Z}.$

We arrive at an overall usable range of $Z_{max} = \min\{Z_{max disparity}, Z_{max error}, Z_{max size}\}$

In [31]:
# basic utils for range estimation
def zmax_from_disparity(f_px: float, B: float, d_min: float) -> float:
    """
    Max range limited by minimum resolvable disparity d_min (in pixels):
    Z_max = f_px * B / d_min
    """
    if any(v <= 0 for v in (f_px, B, d_min)):
        raise ValueError("f_px, B, d_min must be positive.")
    return f_px * B / d_min

def zmax_from_error(f_px: float, B: float, sigma_x: float, sigmaZ_target: float) -> float:
    """
    Max range where depth std equals target (beyond this, error > target):
    sigma_Z = (Z^2 / (f_px * B)) * sqrt(2) * sigma_x
    => Z_max = sqrt( (sigmaZ_target * f_px * B) / (sqrt(2) * sigma_x) )
    """
    if any(v <= 0 for v in (f_px, B, sigma_x, sigmaZ_target)):
        raise ValueError("f_px, B, sigma_x, sigmaZ_target must be positive.")
    from math import sqrt
    return sqrt((sigmaZ_target * f_px * B) / (sqrt(2.0) * sigma_x))

def zmax_from_apparent_size(f_px: float, S: float, s_min_px: float = 3.0) -> float:
    """
    Max range to keep an object of physical size S (meters) at least s_min_px pixels wide/tall:
    s_px = f_px * S / Z  =>  Z_max = f_px * S / s_min_px
    """
    if any(v <= 0 for v in (f_px, S, s_min_px)):
        raise ValueError("f_px, S, s_min_px must be positive.")
    return f_px * S / s_min_px

def sigmaZ_at(f_px, B, Z, sigma_x):
    # Depth std (meters) at depth Z for baseline B and per-camera center noise sigma_x (pixels)
    return (Z**2 / (f_px * B)) * np.sqrt(2.0) * sigma_x

def zmax_combined(f_px: float,
                  B: float,
                  d_min: float | None = None,
                  sigma_x: float | None = None,
                  sigmaZ_target: float | None = None,
                  S: float | None = None,
                  s_min_px: float = 3.0) -> dict:
    """
    Combine applicable constraints and return the limiting Z_max and breakdown.
    Provide any subset of constraints:
      - disparity: d_min
      - accuracy: (sigma_x and sigmaZ_target)
      - apparent size: S (physical size in meters) and s_min_px (pixels, default 3)
    Returns dict with keys: 'Z_max', 'by', and individual components if provided.
    """
    candidates = []
    details = {}

    if d_min is not None:
        Zd = zmax_from_disparity(f_px, B, d_min)
        candidates.append(('disparity', Zd))
        details['Z_disparity'] = Zd

    if (sigma_x is not None) and (sigmaZ_target is not None):
        Ze = zmax_from_error(f_px, B, sigma_x, sigmaZ_target)
        candidates.append(('error', Ze))
        details['Z_error'] = Ze

    if S is not None:
        Zs = zmax_from_apparent_size(f_px, S, s_min_px)
        candidates.append(('object size', Zs))
        details['Z_size'] = Zs

    if not candidates:
        raise ValueError("Provide at least one constraint (d_min, or sigma_x & sigmaZ_target, or S).")

    by, Z_max = min(candidates, key=lambda kv: kv[1])
    details['Z_max'] = Z_max
    details['by'] = by
    return details

def get_hfov(Nx, f_px):
    return np.degrees(2 * np.arctan(Nx/(2*f_px)))

In [34]:
# usage to calculate data for this camera module:
# https://www.farnell.com/datasheets/3822551.pdf

# image pixel count for 1080p, assumes no binning from the 4k image
Nx = 4608
f_mm = 4.74
p_um = 1.4

# distance between cameras in meters
B = 0.3

# min resolvable disparity between images to estimate distance
d_min = 20

# depth error tolerance in meters
sigmaZ_target = 5.0

# drone phyiscal size in meters/pixels
# we can pick a small number here because once we detect a drone within a 30x30px box, we can continue to track its center
# even if it moves away. But if the center size drops below s_min_pix, we cant track it anymore
S = 0.15
s_min_px = 16.0 # say we need drone to be 30 pixels for YOLO to detect it

# thresholds for the deviation in bounding box center
# lo -> more accurate prediction assumption, hi -> less accurate prediction assumption
# pixel uncertainty from bounding box center
sigma_x_lo = 0.2 # pixels
sigma_x_hi = 0.5

# sigmas = np.arange(0.1, 0.5, 0.01)
sigmas = [0.2, 0.5, 1.0, 1.5, 2.0]

# assuming no binning or downscaling
f_px = fx_from_datasheet(f_mm, p_um, binning=1.0, downscale=1.0)

# for each sigma_x, compute:
#   1. max usable detection distance under all constraints
#   2. limiting constraint
#   3. disparity at zmax
#   4 if disparaty limit is greater than minimum
#   5. if the disparity shift between cameras lets the object still fit within the frame
# ... added more

rows = []
for sigma_x in sigmas:
    out = zmax_combined(f_px, B,
                        d_min=d_min,
                        sigma_x=sigma_x, sigmaZ_target=sigmaZ_target,
                        S=S, s_min_px=s_min_px)
    Zmax = out['Z_max']
    by = out['by']
    d_at_Zmax = disparity_expected(f_px, B, Zmax)
    sanity = disparity_sanity(d_at_Zmax, Nx=Nx)
    sigmaZ_at_Zmax = sigmaZ_at(f_px, B, Zmax, sigma_x)
    s_px_at_Zmax = (f_px * S)/Zmax
    rows.append({
        "sigma_x(px)": sigma_x,
        "f_px(px)": round(f_px,1),
        "Baseline B(m)": B,
        "Zmax(m)": Zmax,
        "Limiting constraint": by,
        "Disparity at Zmax (px)": d_at_Zmax,
        # "Enough for matching?": sanity["enough_for_matching"],
        # "Fits in frame?": sanity["fits_in_frame"],
        "sigma_Z at Zmax (m)" : sigmaZ_at_Zmax,
        "size at Zmax (px)": s_px_at_Zmax
    })
    
df_results = pd.DataFrame(rows)
print(f"HFOV: {get_hfov(Nx, f_px)}")
print(df_results)

HFOV: 68.47107014501319
   sigma_x(px)  f_px(px)  Baseline B(m)    Zmax(m) Limiting constraint  \
0          0.2    3385.7            0.3  31.741071         object size   
1          0.5    3385.7            0.3  31.741071         object size   
2          1.0    3385.7            0.3  31.741071         object size   
3          1.5    3385.7            0.3  31.741071         object size   
4          2.0    3385.7            0.3  31.741071         object size   

   Disparity at Zmax (px)  sigma_Z at Zmax (m)  size at Zmax (px)  
0                    32.0             0.280554               16.0  
1                    32.0             0.701385               16.0  
2                    32.0             1.402770               16.0  
3                    32.0             2.104156               16.0  
4                    32.0             2.805541               16.0  


In [33]:
# usage to calculate data for this camera module:
# https://www.newark.com/raspberry-pi/rpi-hq-camera/high-quality-camera-12-3mp/dp/67AH5587?CMP=KNC-GUSA-GEN-PMAX-Shopping-New&mckv=_dc|pcrid||plid||kword||match||slid||product|67AH5587|pgrid||ptaid||&gad_source=1&gad_campaignid=21110543169&gbraid=0AAAAAD5U_g0eKNEywW1ehzPWRtAwXadiU&gclid=Cj0KCQjwovPGBhDxARIsAFhgkwQ2bmOEMqkWmZ7yxTPFxPS2yr3Cvppj6WHbx-Szx9fc78uN5bCMynkaAuk8EALw_wcB
# with this lens
# https://www.arducam.com/arducam-cs-mount-lens-for-raspberry-pi-hq-camera-8mm-focal-length-with-manual-focus.html

Nx = 4056
f_mm = 8.0 # spec has 2 focal lengths listed
p_um = 1.55

B = 0.3

d_min = 20

sigmaZ_target = 5.0

S = 0.15
s_min_px = 16.0 

sigma_x_lo = 0.2 # pixels
sigma_x_hi = 0.5

sigmas = [0.2, 0.5, 1.0, 1.5, 2.0]

f_px = fx_from_datasheet(f_mm, p_um, binning=1.0, downscale=1.0)

rows = []
for sigma_x in sigmas:
    out = zmax_combined(f_px, B,
                        d_min=d_min,
                        sigma_x=sigma_x, sigmaZ_target=sigmaZ_target,
                        S=S, s_min_px=s_min_px)
    Zmax = out['Z_max']
    by = out['by']
    d_at_Zmax = disparity_expected(f_px, B, Zmax)
    sanity = disparity_sanity(d_at_Zmax, Nx=Nx)
    sigmaZ_at_Zmax = sigmaZ_at(f_px, B, Zmax, sigma_x)
    s_px_at_Zmax = (f_px * S)/Zmax
    rows.append({
        "sigma_x(px)": sigma_x,
        "f_px(px)": round(f_px,1),
        "Baseline B(m)": B,
        "Zmax(m)": Zmax,
        "Limiting constraint": by,
        "Disparity at Zmax (px)": d_at_Zmax,
        # "Enough for matching?": sanity["enough_for_matching"],
        # "Fits in frame?": sanity["fits_in_frame"],
        "sigma_Z at Zmax (m)" : sigmaZ_at_Zmax,
        "size at Zmax (px)": s_px_at_Zmax
    })
    
df_results = pd.DataFrame(rows)
print(f"HFOV: {get_hfov(Nx, f_px)}")
print(df_results)

HFOV: 42.902208709624645
   sigma_x(px)  f_px(px)  Baseline B(m)    Zmax(m) Limiting constraint  \
0          0.2    5161.3            0.3  48.387097         object size   
1          0.5    5161.3            0.3  48.387097         object size   
2          1.0    5161.3            0.3  48.387097         object size   
3          1.5    5161.3            0.3  48.387097         object size   
4          2.0    5161.3            0.3  48.387097         object size   

   Disparity at Zmax (px)  sigma_Z at Zmax (m)  size at Zmax (px)  
0                    32.0             0.427686               16.0  
1                    32.0             1.069214               16.0  
2                    32.0             2.138428               16.0  
3                    32.0             3.207642               16.0  
4                    32.0             4.276856               16.0  


### Optimization

In [71]:
def z_size_from(f_px, S, s_min_px):
    return f_px * S / s_min_px

def sigmaZ_at(f_px, B, Z, sigma_x):
    """
    Depth std at depth Z (meters) for baseline B (meters) and per-camera center noise sigma_x (pixels):
      sigma_Z = (Z^2 / (f_px * B)) * sqrt(2) * sigma_x
    """
    return (Z**2 / (f_px * B)) * np.sqrt(2.0) * sigma_x

def B_err_eq_disp(f_px, d_min, sigmaZ_target, sigma_x):
    # error = disparity
    return (sigmaZ_target * d_min**2) / (np.sqrt(2.0) * sigma_x * f_px)

def B_err_eq_size(f_px, S, s_min_px, sigmaZ_target, sigma_x):
    # error = size  (using Z_size = f_px*S/s_min_px)
    Z_size = z_size_from(f_px, S, s_min_px)
    return (Z_size**2 * np.sqrt(2.0) * sigma_x) / (sigmaZ_target * f_px)

def B_disp_eq_size(f_px, d_min, S, s_min_px):
    # disparity at Z_size equals d_min: f_px*B/Z_size = d_min  ⇒  B = Z_size*d_min/f_px
    Z_size = z_size_from(f_px, S, s_min_px)
    return Z_size * d_min / f_px

def Z_at_err_eq_disp(f_px, d_min, sigmaZ_target, sigma_x):
    # Z where error=disparity (same at that B)
    B = B_err_eq_disp(f_px, d_min, sigmaZ_target, sigma_x)
    # Either expression is fine; use disparity form:
    return (f_px * B) / d_min

def optimize_baseline(f_px, d_min, sigmaZ_target, S, s_min_px, sigma_x):
    """
    Return the baseline B_opt that maximizes Z_max(B) and the active limiting constraints.
    Also returns sigma_Z_opt (depth std at the optimum), disparity at the optimum, and image size (px).
    """
    Z_size = z_size_from(f_px, S, s_min_px)

    # Candidate A: equalizing disparity and error
    B_de = B_err_eq_disp(f_px, d_min, sigmaZ_target, sigma_x)
    Z_de = Z_at_err_eq_disp(f_px, d_min, sigmaZ_target, sigma_x)

    if Z_de <= Z_size:
        # Optimum is at disparity = error
        B_opt = B_de
        Z_opt = Z_de
        d_at_opt = f_px * B_opt / Z_opt          # (equals d_min here)
        s_px_opt = f_px * S / Z_opt
        sigma_Z_opt = sigmaZ_at(f_px, B_opt, Z_opt, sigma_x)  # (equals sigmaZ_target here)
        return {
            "B_opt": B_opt,
            "Z_opt": Z_opt,
            "sigma_Z_opt": sigma_Z_opt,
            "d_at_opt": d_at_opt,
            "s_px_opt": s_px_opt,
            "limiting_at_opt": "disparity = error",
            "details": {
                "B_err_eq_disp": B_de,
                "Z_at_err_eq_disp": Z_de,
                "Z_size": Z_size
            }
        }

    # Otherwise size caps the max range; reach Z_size with the smallest B satisfying both constraints
    B_es = B_err_eq_size(f_px, S, s_min_px, sigmaZ_target, sigma_x)  # error >= size
    B_ds = B_disp_eq_size(f_px, d_min, S, s_min_px)                  # disparity >= d_min at Z_size
    B_opt = max(B_es, B_ds)
    Z_opt = Z_size
    d_at_opt = f_px * B_opt / Z_opt
    s_px_opt = f_px * S / Z_opt    # equals s_min_px
    sigma_Z_opt = sigmaZ_at(f_px, B_opt, Z_opt, sigma_x)  # == sigmaZ_target if B_opt == B_es; else < sigmaZ_target

    # Which constraint is tight at B_opt?
    tight = []
    if abs(B_opt - B_es) <= 1e-12: tight.append("error = size")
    if abs(B_opt - B_ds) <= 1e-12: tight.append("disparity = size")
    limiting = " & ".join(tight) if tight else "size-limited (both satisfied)"

    return {
        "B_opt": B_opt,
        "Z_opt": Z_opt,
        "sigma_Z_opt": sigma_Z_opt,
        "d_at_opt": d_at_opt,
        "s_px_opt": s_px_opt,
        "limiting_at_opt": limiting,
        "details": {
                "B_err_eq_size": B_es,
                "B_disp_eq_size": B_ds,
                "Z_size": Z_size
        }
    }

Nx = 4608
f_mm = 7.0
p_um = 1.4

# distance between cameras in meters
B = 0.3

# min resolvable disparity between images to estimate distance
d_min = 20

# depth error tolerance in meters
sigmaZ_target = 5.0

# drone phyiscal size in meters/pixels
# we can pick a small number here because once we detect a drone within a 30x30px box, we can continue to track its center
# even if it moves away. But if the center size drops below s_min_pix, we cant track it anymore
S = 0.15
s_min_px = 16.0 # say we need drone to be 30 pixels for YOLO to detect it

# thresholds for the deviation in bounding box center
# lo -> more accurate prediction assumption, hi -> less accurate prediction assumption
# pixel uncertainty from bounding box center
sigma_x_lo = 0.2 # pixels
sigma_x_hi = 0.5

# sigmas = np.arange(0.1, 0.5, 0.01)
sigmas = [0.2, 0.5, 1.0, 1.5, 2.0]

# sigmas = np.arange(0.01, 0.5, 0.01)
# sigmas = [0.2, 0.5, 1.0, 1.5, 2.0]

def fx_from_datasheet(f_mm: float, p_um: float, *, binning: float = 1.0, downscale: float = 1.0) -> float:
    p_mm = p_um / 1000.0
    return (f_mm / p_mm) / (binning * downscale)

f_px = fx_from_datasheet(f_mm, p_um, binning=1.0, downscale=1.0)

for sigma_x in sigmas:
    res = optimize_baseline(f_px, d_min, sigmaZ_target, S, s_min_px, sigma_x)
    print(f"\n--- sigma_x = {sigma_x} px ---")
    print(f"B_opt        = {res['B_opt']:.6f} m")
    print(f"Z_opt        = {res['Z_opt']:.3f} m")
    print(f"sigma_Z_opt  = {res['sigma_Z_opt']:.4f} m")
    print(f"d_at_opt     = {res['d_at_opt']:.2f} px")
    print(f"s_px_opt     = {res['s_px_opt']:.2f} px")
    print(f"Limiting @opt= {res['limiting_at_opt']}")
    for k, v in res["details"].items():
        print(f"{k}: {v}")


--- sigma_x = 0.2 px ---
B_opt        = 0.187500 m
Z_opt        = 46.875 m
sigma_Z_opt  = 0.6629 m
d_at_opt     = 20.00 px
s_px_opt     = 16.00 px
Limiting @opt= disparity = size
B_err_eq_size: 0.024859222776089564
B_disp_eq_size: 0.1875
Z_size: 46.875

--- sigma_x = 0.5 px ---
B_opt        = 0.187500 m
Z_opt        = 46.875 m
sigma_Z_opt  = 1.6573 m
d_at_opt     = 20.00 px
s_px_opt     = 16.00 px
Limiting @opt= disparity = size
B_err_eq_size: 0.062148056940223906
B_disp_eq_size: 0.1875
Z_size: 46.875

--- sigma_x = 1.0 px ---
B_opt        = 0.187500 m
Z_opt        = 46.875 m
sigma_Z_opt  = 3.3146 m
d_at_opt     = 20.00 px
s_px_opt     = 16.00 px
Limiting @opt= disparity = size
B_err_eq_size: 0.12429611388044781
B_disp_eq_size: 0.1875
Z_size: 46.875

--- sigma_x = 1.5 px ---
B_opt        = 0.187500 m
Z_opt        = 46.875 m
sigma_Z_opt  = 4.9718 m
d_at_opt     = 20.00 px
s_px_opt     = 16.00 px
Limiting @opt= disparity = size
B_err_eq_size: 0.18644417082067172
B_disp_eq_size: 0.1875
Z

### How is B being optimized
At low B, the cameras are close, the disparity between images is low. We dont get enough shift between cameras to do distance estimation.

At high B, the cameras are far and the size of the drone in the image will become the limiting factor before we run out of pixel difference between images to classify the distance.

There is an intersection between these two where the disparity = the error we are aiming for in our distance estimate. 